In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue May 18 14:31:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'  'My Drive'   yolov3


In [3]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [4]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [5]:
# Compile
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1139:10: note: ...thi

In [6]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [14]:
!echo -e 'Not Wearing Mask\n Wearing Mask \n Please Wear Mask Correctly' > data/obj.names
!echo -e 'classes= 3 \ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [15]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

In [16]:
!mkdir data/obj
!unzip /mydrive/yolov3/Dataset.zip -d data/obj

mkdir: cannot create directory ‘data/obj’: File exists
Archive:  /mydrive/yolov3/Dataset.zip
replace data/obj/Dataset/124427226_426881011826518_1113011506116592785_n.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/Dataset/124427226_426881011826518_1113011506116592785_n.jpg  
 extracting: data/obj/Dataset/124427226_426881011826518_1113011506116592785_n.txt  
  inflating: data/obj/Dataset/124553393_4160893490605139_2633653380416795852_n.jpg  
 extracting: data/obj/Dataset/124553393_4160893490605139_2633653380416795852_n.txt  
  inflating: data/obj/Dataset/131209871_741569846442290_2955525421274824177_n.jpg  
  inflating: data/obj/Dataset/131209871_741569846442290_2955525421274824177_n.txt  
  inflating: data/obj/Dataset/155001616_491025852321039_5316197695371108200_n.jpg  
  inflating: data/obj/Dataset/155001616_491025852321039_5316197695371108200_n.txt  
  inflating: data/obj/Dataset/1621167446518.jpg  
 extracting: data/obj/Dataset/1621167446518.txt  
  inflating: da

In [19]:
import glob
images_list = glob.glob("data/obj/Dataset/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [20]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-05-18 14:39:38--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.2’

darknet53.conv.74.2 100%[===================>] 154.96M  6.64MB/s    in 18s     

2021-05-18 14:39:56 (8.81 MB/s) - ‘darknet53.conv.74.2’ saved [162482580/162482580]



In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74.2 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000015, iou_loss = 0.000000, total_loss = 0.000015 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 75503, rewritten_bbox = 0.037085 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.796418), count: 5, class_loss = 0.440051, iou_loss = 0.199225, total_loss = 0.639277 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000012, iou_loss = 0.000000, total_loss = 0.000012 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 75508, rewritten_bbox = 0.037082 % 
v3 (mse